
# Demonstração de Uso de Ferramentas para Manipulação de Modelos de Linguagem

Nesta seção, aplicamos os conceitos apresentados nas seções anteriores em uma demonstração prática. Vamos explorar o uso de um modelo de linguagem generativa pré-treinado, desde a inicialização até a geração de texto, com exemplos claros e objetivos.

**Objetivo da demonstração:**
* Mostrar como carregar e usar um modelo pré-treinado da biblioteca Hugging Face.
* Gerar texto a partir de prompts personalizados.
* Explorar como diferentes configurações de parâmetros impactam os resultados.
* Ao final, você será capaz de utilizar modelos de linguagem para tarefas práticas como geração de texto criativo, resumos ou respostas para perguntas específicas.








# Instalar a biblioteca Transformers (caso não esteja instalada)

In [ ]:
!pip install transformers

# Carregando o Modelo

# Carrega o Modelo e faz teste básico de prompt


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset

# Carrega modelo e tokenizador
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.config.loss_type = "ForCausalLMLoss"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Define explicitamente

# Teste antes do fine-tuning
prompt = "What is IFSC-Joinville?"
inputs_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)  # Move para GPU
outputs = model.generate(**inputs_prompt, max_length=50)

print(f"\n\nAntes do fine-tuning: {tokenizer.decode(outputs[0], skip_special_tokens=True)}\n\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Antes do fine-tuning: What is IFSC-Joinville?

IFSC-Joinville is a community-based organization that provides a safe, safe, and supportive environment for all members of the community. We are a non-profit organization that provides a safe,




# Prepara Base de Treinamento e efetua o Treinamento

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

class SimpleDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: tensor[idx] for key, tensor in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return self.encodings["input_ids"].size(0)

# Dados de treino
texts = [
    "Question: What is IFSC-Joinville? Answer: IFSC-Joinville is the Federal Institute of Santa Catarina - Joinville Campus, a public educational institution in Brazil.",
    "Question: Where is IFSC-Joinville located? Answer: IFSC-Joinville is located in the city of Joinville, in the state of Santa Catarina, Brazil.",
    "Question: What type of institution is IFSC-Joinville? Answer: IFSC-Joinville is a public federal institution focused on technical and technological education.",
    "Question: What courses does IFSC-Joinville offer? Answer: IFSC-Joinville offers technical, undergraduate, and graduate courses in various fields such as engineering, technology, and management.",
    "Question: Is IFSC-Joinville a public or private institution? Answer: IFSC-Joinville is a public institution.",
    "Question: What is the mission of IFSC-Joinville? Answer: The mission of IFSC-Joinville is to provide quality education, promote research, and contribute to the development of society.",
    "Question: Who can study at IFSC-Joinville? Answer: IFSC-Joinville is open to all students who pass the selection process for its courses.",
    "Question: Does IFSC-Joinville charge tuition fees? Answer: No, IFSC-Joinville is a public institution and does not charge tuition fees.",
    "Question: What is the main language of instruction at IFSC-Joinville? Answer: The main language of instruction at IFSC-Joinville is Portuguese.",
    "Question: When was IFSC-Joinville founded? Answer: IFSC-Joinville was established as part of the Federal Institute of Santa Catarina in 2008.",
    "Question: What is the address of IFSC-Joinville? Answer: The address of IFSC-Joinville is Av. Getúlio Vargas, 830, Joinville, Santa Catarina, Brazil.",
    "Question: Does IFSC-Joinville offer distance learning courses? Answer: Yes, IFSC-Joinville offers some courses in distance learning mode.",
    "Question: What are the research areas at IFSC-Joinville? Answer: IFSC-Joinville conducts research in areas such as engineering, technology, environment, and education.",
    "Question: How can I apply to IFSC-Joinville? Answer: You can apply to IFSC-Joinville by participating in the selection process announced on their official website.",
    "Question: Does IFSC-Joinville have partnerships with companies? Answer: Yes, IFSC-Joinville maintains partnerships with companies and institutions for internships and research projects.",
    "Question: What facilities are available at IFSC-Joinville? Answer: IFSC-Joinville offers laboratories, libraries, study areas, and sports facilities.",
    "Question: Who is the director of IFSC-Joinville? Answer: The director of IFSC-Joinville is appointed by the Federal Institute of Santa Catarina and may change over time.",
    "Question: Does IFSC-Joinville offer scholarships? Answer: IFSC-Joinville offers scholarships for research, extension, and student support programs.",
    "Question: What is the website of IFSC-Joinville? Answer: The official website of IFSC-Joinville is https://www.ifsc.edu.br/joinville.",
    "Question: What is the phone number of IFSC-Joinville? Answer: The phone number of IFSC-Joinville is +55 (47) 3441-1400."
]
inputs = tokenizer(
    texts,
    return_tensors="pt",
    max_length=512,
    truncation=True,
    padding=True
)
train_dataset = SimpleDataset(inputs)

# Fine-tuning
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=15,  # Aumente para 10-15 epochs
    per_device_train_batch_size=2,  # Reduza batch size para evitar overfitting
    learning_rate=3e-5,  # Taxa de aprendizado menor
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    logging_dir="./logs",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.015300


TrainOutput(global_step=150, training_loss=0.7774440256754557, metrics={'train_runtime': 431.7196, 'train_samples_per_second': 0.695, 'train_steps_per_second': 0.347, 'total_flos': 6889536000000.0, 'train_loss': 0.7774440256754557, 'epoch': 15.0})

# Testa o modelo Treinado

In [ ]:
# Teste depois do fine-tuning
prompt = "What is IFSC-Joinville?"
inputs_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)  # Move para GPU

outputs = model.generate(
    **inputs_prompt.to(model.device),
    max_length=80,
    num_return_sequences=1
)  # Garante dispositivo

print(f"\n\nDepois do fine-tuning: {tokenizer.decode(outputs[0], skip_special_tokens=True)}\n")

# Teste depois do fine-tuning
prompt = "What courses does IFSC-Joinville offer?"
inputs_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)  # Move para GPU

outputs = model.generate(
    **inputs_prompt.to(model.device),
    max_length=120,
    num_return_sequences=1
)  # Garante dispositivo

print(f"\nDepois do fine-tuning (2): {tokenizer.decode(outputs[0], skip_special_tokens=True)}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Depois do fine-tuning: What is IFSC-Joinville? Answer: IFSC-Joinville is a public educational institution located in the city of Joinville, in the state of Santa Catarina, Brazil. The institution is open to all students who pass the entrance exam. The institution is open to all students who pass the selection process for its courses. The institution also offers internships and research opportunities. The institution


Depois do fine-tuning (2): What courses does IFSC-Joinville offer? Yes, IFSC-Joinville offers internships in various fields such as engineering, technology, and management. Internships in these fields include technical, technical, and management. The courses offered by IFSC-Joinville are open to all students who pass the selection process of participating in the selection process announced on their official website. What courses does IFSC-Joinville offer? IFSC-Joinville offers courses in various fields such as engineering, technology, and management. Internships in thes